# Generate a balanced session from a lapseless session
**Use case:** Given a session with a high number of motion images, but an insufficient number of lapse images (i.e. only daily lapse images)

In [16]:
%load_ext autoreload
%autoreload 2

import shutil
import os
from tqdm import tqdm
from py.Dataset import Dataset
from py.Session import Session, MotionImage, LapseImage
from py.Labels import LABELS

DIR = '/home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/'
TARGET_DIR = "./GeneratedSessions_NoBackup/"

ds = Dataset(DIR)
target_session = ds.create_session("fox_03")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 32 sessions
Session 'Fox_03' at folder: /home/AMMOD_data/camera_traps/BayerWald/Vielkadaver-Projekt/VIELAAS_Spring_Session03-VIELAAS_Fox_03
Loaded scans.


We will pick capture sets where every image is labeled as normal, until we have selected at least pick_for_lapse images.

In [38]:
pick_for_lapse = 1200

normal_list = LABELS[target_session.name]["normal"]
print(f"Picking at least {pick_for_lapse} of {len(normal_list)} images")
assert pick_for_lapse < len(normal_list)

lapse_img_nrs = []
while len(lapse_img_nrs) < pick_for_lapse:
    img_set = target_session.get_random_motion_image_set()
    set_nrs = []
    for img in img_set:
        img_nr = int(img.filename[-9:-4])
        if img_nr not in normal_list:
            # at least one image in set is anomalous, discard whole set
            set_nrs = []
            break
        elif img_nr not in lapse_img_nrs:
            set_nrs.append(img_nr)
    # whole set contains empty images, so add the whole set
    lapse_img_nrs.extend(set_nrs)
print(lapse_img_nrs)

Picking at least 1200 of 3702 images
[2067, 2068, 2069, 2070, 2066, 2059, 2060, 2058, 2056, 2057, 2053, 2054, 2055, 2051, 2052, 2046, 2050, 2047, 2049, 2048, 2063, 2064, 2062, 2065, 2061, 542, 544, 543, 545, 541, 532, 531, 533, 534, 535, 540, 536, 537, 539, 538, 529, 527, 528, 526, 530, 5359, 5356, 5357, 5358, 5360, 5348, 5349, 5346, 5350, 5347, 5353, 5354, 5355, 5352, 5351, 5333, 5335, 5332, 5331, 5334, 5365, 5362, 5361, 5364, 5363, 5329, 5326, 5327, 5328, 5330, 5342, 5345, 5344, 5341, 5343, 5339, 5338, 5340, 5336, 5337, 4826, 4829, 4828, 4827, 4830, 4809, 4807, 4810, 4808, 4806, 4817, 4816, 4820, 4819, 4818, 4832, 4835, 4833, 4834, 4831, 4825, 4822, 4824, 4821, 4823, 4811, 4813, 4815, 4814, 4812, 227, 230, 226, 228, 229, 253, 251, 254, 252, 255, 221, 223, 225, 222, 224, 211, 215, 212, 214, 213, 205, 202, 203, 201, 204, 260, 256, 257, 259, 258, 236, 240, 238, 239, 237, 248, 247, 250, 249, 246, 208, 207, 210, 206, 209, 233, 235, 231, 234, 232, 220, 217, 216, 218, 219, 244, 245, 243, 24

In [39]:
len(lapse_img_nrs)

1210

We will now copy the Motion images to either Lapse or Motion, depending on whether they are in lapse_img_nrs.

In [40]:
# copy motion images to Motion and Lapse folders
motion_folder = os.path.join(TARGET_DIR, os.path.basename(target_session.folder), "Motion")
lapse_folder = os.path.join(TARGET_DIR, os.path.basename(target_session.folder), "Lapse")
os.makedirs(motion_folder, exist_ok=True)
os.makedirs(lapse_folder, exist_ok=True)
lapse = 0
motion = 0
for motion_img in tqdm(list(target_session.generate_motion_images())):
    img_nr = int(motion_img.filename[-9:-4])
    if img_nr in lapse_img_nrs:
        shutil.copy(motion_img.get_full_path(), lapse_folder)
        lapse += 1
    else:
        shutil.copy(motion_img.get_full_path(), motion_folder)
        motion += 1
print(f"Copied {lapse} files to Lapse, {motion} to Motion")

100%|██████████| 5495/5495 [03:08<00:00, 29.08it/s]

Copied 1210 files to Lapse, 4285 to Motion


The full folder is copied without changes.

In [41]:
# copy full folder as-is
full_folder = os.path.join(TARGET_DIR, os.path.basename(target_session.folder), "Full")
shutil.copytree(target_session.get_full_folder(), full_folder)

'./GeneratedSessions_NoBackup/VIELAAS_Spring_Session03-VIELAAS_Fox_03/Full'